# Distribution (Generalization performance by TRAIN vs TEST)

In [ ]:
import pandas as pd

df = pd.DataFrame(index=[idx for idx in range(20)])

with open("../scripts/RUN_221213_DIST/res.txt") as f:
    ls = f.readlines()
    for line in ls:
        ds = line.split("\t")

        vals = eval(ds[1])

        col_rmse = ds[0]+"_rmse"
        col_r2 = ds[0]+"_r2"

        df.loc[:, col_rmse] = pd.Series([val[0] for val in vals])
        df.loc[:, col_r2] = pd.Series([val[1] for val in vals])


In [ ]:
import matplotlib.pyplot as plt

mmss = [f"{idx:02}" for idx in range(1, 16, 1)]
mtxs = ["rmse", "r2"]

for mms in mmss:

    print(mms)
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    for idx, mtx in enumerate(mtxs):
        col_train = f"{mms}_train_{mtx}"
        col_test = f"{mms}_test_{mtx}"

        if col_train not in df.columns or col_test not in df.columns:
            continue
        
        x1 = df[f"{mms}_train_{mtx}"]
        x2 = df[f"{mms}_test_{mtx}"]

        axes[idx].hist([x1, x2], bins=10, label=[f"MMS{mms}_FV_{mtx}_train", f"MMS{mms}_FV_{mtx}_test"])

        axes[idx].legend()

    plt.show()
# df[df.columns.str.match("^01_test_r2")[0]].hist(alpha=0.5)

# GP (no filters)

In [ ]:
import pandas as pd


res_dict = {
    "N": "../scripts/RUN_230203_N/res.txt",
    "N_X.1": "../scripts/RUN_230203_N1_1/res.txt",
    "N_C.1": "../scripts/RUN_230203_N2_1/res.txt",
    "FV": "../scripts/RUN_230203_FV/res.txt",
    # "FV_aic": "../scripts/RUN_230206_A_FV/res.txt",
    # "FV_bic": "../scripts/RUN_230206_B_FV/res.txt",
    # "FV_caic": "../scripts/RUN_230206_C_FV/res.txt",
    # "FV_X0": "../scripts/RUN_230203_FV10_1/res.txt",
    # "FV_C0": "../scripts/RUN_230203_FV20_1/res.txt",
    ###
    "FV_X.5": "../scripts/RUN_230203_FV1_5/res.txt",
    "FV_X.1": "../scripts/RUN_230203_FV1_1/res.txt",
    "FV_X.01": "../scripts/RUN_230203_FV1_01/res.txt",
    "FV_X01.1": "../scripts/RUN_230203_FV11_1/res.txt",
    ###
    "FV_C.1": "../scripts/RUN_230203_FV2_1/res.txt",
    "FV_C.01": "../scripts/RUN_230203_FV2_01/res.txt",
    "FV_C.001": "../scripts/RUN_230203_FV2_001/res.txt",
    "FV_C01.1": "../scripts/RUN_230203_FV21_1/res.txt",
    ###
    "FVD": "../scripts/RUN_230123_FVD/res.txt",
    "FVD_X.1": "../scripts/RUN_230203_FVD1_1/res.txt",
    ###
    "SVR": "../scripts/res_svr.txt",
    "MLR": "../scripts/res_mlr.txt"
}

df = pd.DataFrame()

for method, resfile in res_dict.items():

    print(method, resfile)
    with open(resfile) as f:
        ls = f.readlines()
        # print(ls)
        for line in ls:
            ds = line.split("\t")

            if "train_all" in ds[0]:
                continue

            # XXXXXX
            _val = ds[1].replace("inf", "10.0", 10)
            vals = eval(_val)
            # vals = eval(ds[1])

            row_rmse = "MMS"+ds[0]+"_rmse"
            row_r2 = "MMS"+ds[0]+"_r2"

            df.loc[row_rmse, method] = vals[0]
            df.loc[row_r2, method] = vals[1]


# df.index

In [ ]:
import matplotlib.pyplot as plt

mmss = [f"{idx:02}" for idx in range(1, 16, 1)]
mtxs = ["rmse", "r2"]

DO_SCATTER=False

for mms in mmss:

    print(mms)
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    for idx, mtx in enumerate(mtxs):
        row_train = f"MMS{mms}_train_{mtx}"
        row_test = f"MMS{mms}_test_{mtx}"

        if row_train not in df.index or row_test not in df.index:
            continue

        _df = df.loc[[row_train,row_test],:].T

        if DO_SCATTER:
            # x1 = df[f"{mms}_train_{mtx}"]
            # x2 = df[f"{mms}_test_{mtx}"]

            _df.plot.scatter(x=row_train, y=row_test, ax=axes[idx], c=[_j for _j in range(len(list(_df.index)))], alpha=0.8, s=100)

            for _jdx, _label in enumerate(list(_df.index)):
                axes[idx].text(_df.iloc[_jdx,0], _df.iloc[_jdx,1], _label, alpha=0.75)
            
            _xlim = axes[idx].get_xlim()
            _ylim = axes[idx].get_ylim()
            axes[idx].set_xlim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
            axes[idx].set_ylim(min(_xlim[0],_ylim[0]), max(_xlim[1],_ylim[1]))
            _xlim = axes[idx].get_xlim()
            _ylim = axes[idx].get_ylim()
            axes[idx].plot(_xlim, _ylim, color="black", lw=.5)
        else:
            _df.T.plot.bar(ax=axes[idx])
            # axes[idx].bar([x1, x2], label=[f"MMS{mms}_FV_{mtx}_train", f"MMS{mms}_FV_{mtx}_test"])


    plt.show()